In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


## Exploring Data

In [3]:
df = pd.read_csv("data/monatszahlen2412_verkehrsunfaelle_06_12_24.csv")